In [1]:
import numpy as np
from scipy import stats
import pandas as pd

In [2]:
# Specify the path to the .npy file
GMU_File = "results/gmu-Arevalo-F1_scores.npy"
SimulParallel_File = "results/simulParallel-Xie-F1_scores.npy"
SMCA_File = "results/SMCA-F1_scores.npy"

# Load the F1 scores from the file
GMU_F1_scores = np.load(GMU_File)
SimulParallel_F1_scores = np.load(SimulParallel_File)
SMCA_F1_scores = np.load(SMCA_File)

# Display the F1 scores
print("GMU Model Cross-Validation F1 scores:", GMU_F1_scores)
print("Simul-Parallel Cross-Validation F1 scores:", SimulParallel_F1_scores)
print("SMCA Model Cross-Validation F1 scores:", SMCA_F1_scores)

GMU Model Cross-Validation F1 scores: [0.75       0.71014488 0.73504275 0.77862602 0.73684216]
Simul-Parallel Cross-Validation F1 scores: [0.         0.06060607 0.04166666 0.0776699  0.04347826]
SMCA Model Cross-Validation F1 scores: [0.2992126  0.33755273 0.3214286  0.18897636 0.24390243]


In [3]:
# Display the F1 scores in list form
print("GMU Model Cross-Validation F1 scores:")
for i, score in enumerate(GMU_F1_scores, start=1):
    print(f"  Fold {i}: {score:.4f}")

print("\nSimul-Parallel Cross-Validation F1 scores:")
for i, score in enumerate(SimulParallel_F1_scores, start=1):
    print(f"  Fold {i}: {score:.4f}")

print("\nSMCA Model Cross-Validation F1 scores:")
for i, score in enumerate(SMCA_F1_scores, start=1):
    print(f"  Fold {i}: {score:.4f}")

GMU Model Cross-Validation F1 scores:
  Fold 1: 0.7500
  Fold 2: 0.7101
  Fold 3: 0.7350
  Fold 4: 0.7786
  Fold 5: 0.7368

Simul-Parallel Cross-Validation F1 scores:
  Fold 1: 0.0000
  Fold 2: 0.0606
  Fold 3: 0.0417
  Fold 4: 0.0777
  Fold 5: 0.0435

SMCA Model Cross-Validation F1 scores:
  Fold 1: 0.2992
  Fold 2: 0.3376
  Fold 3: 0.3214
  Fold 4: 0.1890
  Fold 5: 0.2439


In [4]:
# Create a DataFrame
f1_scores_df = pd.DataFrame({
    'Fold': np.arange(1, len(GMU_F1_scores) + 1),
    'GMU F1 Score': GMU_F1_scores,
    'Simul-Parallel F1 Score': SimulParallel_F1_scores,
    'SMCA F1 Score': SMCA_F1_scores
})
# Display the DataFrame without the index
print(f1_scores_df.to_string(index=False))

 Fold  GMU F1 Score  Simul-Parallel F1 Score  SMCA F1 Score
    1      0.750000                 0.000000       0.299213
    2      0.710145                 0.060606       0.337553
    3      0.735043                 0.041667       0.321429
    4      0.778626                 0.077670       0.188976
    5      0.736842                 0.043478       0.243902


In [5]:

t_stat_SMCA_Simul, p_value_SMCA_Simul = stats.ttest_rel(SMCA_F1_scores, SimulParallel_F1_scores)
print(f"Paired t-test between SMCA Model and Simul-Parallel Model F1 Scores:\n t-statistic = {t_stat_SMCA_Simul:.9f}, p-value = {p_value_SMCA_Simul:.9f}")

alpha = 0.05
if p_value_SMCA_Simul < alpha:
    print("Reject null hypothesis: There is a significant difference between the SMCA Model and Simul-Parallel Model F1 Scores")
else:
    print("Fail to reject the null hypothesis: There is no significant difference between the SMCA Model and Simul-Parallel Model F1 Scores")


Paired t-test between SMCA Model and Simul-Parallel Model F1 Scores:
 t-statistic = 6.689905207, p-value = 0.002596555
Reject null hypothesis: There is a significant difference between the SMCA Model and Simul-Parallel Model F1 Scores


In [6]:
t_stat_SMCA_GMU, p_value_SMCA_GMU = stats.ttest_rel(SMCA_F1_scores, GMU_F1_scores)
print(f"Paired t-test between SMCA Model and GMU Model F1 Scores:\n t-statistic = {t_stat_SMCA_GMU:.9f}, p-value = {p_value_SMCA_GMU:.9f}")

alpha = 0.05 
if p_value_SMCA_GMU < alpha:
    print("Reject null hypothesis: There is a significant difference between the SMCA Model and GMU Model F1 Scores")
else:
    print("Fail to reject the null hypothesis: There is no significant difference between the SMCA Model and GMU Model F1 Scores")

Paired t-test between SMCA Model and GMU Model F1 Scores:
 t-statistic = -12.467000886, p-value = 0.000238068
Reject null hypothesis: There is a significant difference between the SMCA Model and GMU Model F1 Scores
